<a href="https://colab.research.google.com/github/Sarthakpunj/Rag-using-vector_db-and-huugingface/blob/main/RAG_FROM_HUGGING_FACE_AND_VECTOR_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index-llms-huggingface

In [ ]:
!pip install llama-index

In [ ]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader
from llama_index.llms.huggingface import HuggingFaceLLM

In [ ]:
!mkdir data #create a data directory

In [ ]:
#load document
documents = SimpleDirectoryReader('./data/').load_data()

In [ ]:
print(documents)

In [ ]:
from llama_index.core import PromptTemplate #role = assitant(llm), user(us), system(developer env)
system_prompt = """<|SYSTEM|># You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided.
"""

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")


In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    device_map="auto",
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}
)


In [ ]:
#embedding
!pip install llama-index-embeddings-huggingface
!pip install llama-index-embeddings-instructor

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")

In [ ]:
from llama_index.core import vector_stores, ServiceContext

from llama_index.core import Settings

# Configure the settings directly
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 1024

In [ ]:
index = VectorStoreIndex.from_documents(documents)

In [ ]:
query= index.as_query_engine()

In [ ]:
query.query("what is closify?")